# 라이브러리 및 데이터

In [1]:
import random
import os 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, make_scorer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import confusion_matrix
import joblib
import xgboost as xgb
from lightgbm import LGBMRegressor
from catboost import CatBoostClassifier, Pool
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

#시드 고정
DEFAULT_RANDOM_SEED = 3031
def seedBasic(seed=DEFAULT_RANDOM_SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

In [2]:
train = pd.read_csv('train.csv')
train

,X1,X3,X5,X6,X7a,X7b,X7c,X7d,X7e,X7f,...,X72_binary,X73_binary,X74_binary,X75_binary,X76_binary,X77_binary,X78_binary,anomaly,score,class
0,3490,8.000000e+01,0.0,0.0,0.0,0.0,0.0,0.0,179466.0,52702.0,...,0,0,0,0,0,0,0,1,0.113947,0
1,92,1.400000e+01,0.0,0.0,0.0,0.0,0.0,4528.0,3486.0,3848.0,...,0,0,0,0,0,0,0,1,0.131765,0
2,10,1.800000e+01,4.0,6.0,0.0,0.0,0.0,0.0,224.0,1802.0,...,0,0,0,0,0,0,0,1,0.137487,0
3,156758,2.130706e+09,0.0,0.0,0.0,0.0,0.0,0.0,38538.0,2412800.0,...,0,0,0,0,0,0,0,-1,-0.003053,0
4,121048,3.352000e+03,326.0,924.0,0.0,0.0,46.0,490092.0,3101434.0,4041666.0,...,0,0,0,0,0,0,0,-1,-0.108592,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52158,69628,3.320000e+02,0.0,0.0,0.0,0.0,0.0,0.0,44998.0,1580944.0,...,0,0,0,0,0,0,0,1,0.101496,0
52159,23910,8.680000e+02,0.0,0.0,0.0,5932.0,217980.0,1155898.0,1349820.0,224616.0,...,0,0,0,0,0,0,0,1,0.001940,0
52160,14,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,728.0,4008.0,...,0,0,0,0,0,0,0,1,0.137923,0
52161,437486,1.540000e+02,0.0,0.0,0.0,96944.0,981638.0,2445458.0,4387124.0,4934430.0,...,0,0,0,0,0,0,0,-1,-0.173980,1


In [3]:
X = train.drop(['class'],axis=1)
y = train['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42, stratify = y, shuffle = True)

# XGB Classifier_튜닝X

In [4]:
%%time
seedBasic(42)
xgb_model = xgb.XGBClassifier(random_state=42)
xgb_model.fit(X, y)

model = xgb_model.fit(X,y)

CPU times: total: 7min 13s
Wall time: 2min 41s


In [5]:
joblib.dump(model, 'XGBbasic.pkl')

['XGBbasic.pkl']

# XGB Classifier_best튜닝

In [6]:
%%time
seedBasic(42)
xgb_model = xgb.XGBClassifier(scale_pos_weight=(len(y) - y.sum()) / y.sum(), random_state=42)
xgb_model.fit(X, y)

model = xgb_model.fit(X,y)

CPU times: total: 7min 45s
Wall time: 3min 29s


In [7]:
joblib.dump(model, 'XGBbest.pkl')

['XGBbest.pkl']

# LGBM Regressor_튜닝X

In [8]:
%%time
lgb_r = LGBMRegressor(random_state=42)
lgb_r.fit(X, y)

CPU times: total: 11.8 s
Wall time: 5.54 s


LGBMRegressor(random_state=42)

In [9]:
joblib.dump(model, 'lgbrbasic.pkl')

['lgbrbasic.pkl']

# LGBM Regressor_best튜닝

In [10]:
%%time
lgb_r = LGBMRegressor(learning_rate = 0.05, n_estimators=100,
                      num_leaves=50, reg_alpha = 1.5, reg_lambda = 0.5, random_state=200)

lgb_r.fit(X, y)

CPU times: total: 13.4 s
Wall time: 6.26 s


LGBMRegressor(learning_rate=0.05, num_leaves=50, random_state=200,
              reg_alpha=1.5, reg_lambda=0.5)

In [11]:
joblib.dump(model, 'lgbrbest.pkl')

['lgbrbest.pkl']

# CatBoost 

In [12]:
%%time
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y)
class_weights = dict(zip(classes, weights))

cat = CatBoostClassifier(learning_rate=0.5, class_weights=class_weights, random_state=42)
cat.fit(X, y)

0:	learn: 0.2537197	total: 272ms	remaining: 4m 31s
1:	learn: 0.1639602	total: 440ms	remaining: 3m 39s
2:	learn: 0.1345234	total: 616ms	remaining: 3m 24s
3:	learn: 0.1178589	total: 861ms	remaining: 3m 34s
4:	learn: 0.1057333	total: 1.02s	remaining: 3m 23s
5:	learn: 0.0978947	total: 1.27s	remaining: 3m 30s
6:	learn: 0.0897886	total: 1.39s	remaining: 3m 17s
7:	learn: 0.0860487	total: 1.51s	remaining: 3m 7s
8:	learn: 0.0814722	total: 1.61s	remaining: 2m 57s
9:	learn: 0.0760439	total: 1.7s	remaining: 2m 48s
10:	learn: 0.0736720	total: 1.84s	remaining: 2m 45s
11:	learn: 0.0675419	total: 1.95s	remaining: 2m 40s
12:	learn: 0.0629117	total: 2.04s	remaining: 2m 34s
13:	learn: 0.0608582	total: 2.13s	remaining: 2m 29s
14:	learn: 0.0581196	total: 2.24s	remaining: 2m 26s
15:	learn: 0.0548516	total: 2.39s	remaining: 2m 27s
16:	learn: 0.0520879	total: 2.52s	remaining: 2m 25s
17:	learn: 0.0496255	total: 2.62s	remaining: 2m 22s
18:	learn: 0.0479617	total: 2.73s	remaining: 2m 21s
19:	learn: 0.0451240	tot

158:	learn: 0.0052582	total: 17.8s	remaining: 1m 33s
159:	learn: 0.0052581	total: 17.9s	remaining: 1m 33s
160:	learn: 0.0052580	total: 17.9s	remaining: 1m 33s
161:	learn: 0.0052580	total: 18s	remaining: 1m 33s
162:	learn: 0.0051776	total: 18.1s	remaining: 1m 33s
163:	learn: 0.0051520	total: 18.3s	remaining: 1m 33s
164:	learn: 0.0051520	total: 18.4s	remaining: 1m 33s
165:	learn: 0.0051519	total: 18.6s	remaining: 1m 33s
166:	learn: 0.0051087	total: 18.8s	remaining: 1m 33s
167:	learn: 0.0051087	total: 18.9s	remaining: 1m 33s
168:	learn: 0.0049460	total: 19s	remaining: 1m 33s
169:	learn: 0.0047012	total: 19.1s	remaining: 1m 33s
170:	learn: 0.0047012	total: 19.2s	remaining: 1m 33s
171:	learn: 0.0045874	total: 19.3s	remaining: 1m 32s
172:	learn: 0.0045873	total: 19.4s	remaining: 1m 32s
173:	learn: 0.0045006	total: 19.5s	remaining: 1m 32s
174:	learn: 0.0045001	total: 19.6s	remaining: 1m 32s
175:	learn: 0.0044292	total: 19.7s	remaining: 1m 32s
176:	learn: 0.0043662	total: 19.8s	remaining: 1m 3

314:	learn: 0.0043645	total: 33.7s	remaining: 1m 13s
315:	learn: 0.0043645	total: 33.8s	remaining: 1m 13s
316:	learn: 0.0043645	total: 33.8s	remaining: 1m 12s
317:	learn: 0.0043645	total: 33.9s	remaining: 1m 12s
318:	learn: 0.0043645	total: 34s	remaining: 1m 12s
319:	learn: 0.0043645	total: 34.1s	remaining: 1m 12s
320:	learn: 0.0043645	total: 34.2s	remaining: 1m 12s
321:	learn: 0.0043645	total: 34.3s	remaining: 1m 12s
322:	learn: 0.0043645	total: 34.3s	remaining: 1m 11s
323:	learn: 0.0043645	total: 34.4s	remaining: 1m 11s
324:	learn: 0.0043645	total: 34.5s	remaining: 1m 11s
325:	learn: 0.0043645	total: 34.6s	remaining: 1m 11s
326:	learn: 0.0043645	total: 34.6s	remaining: 1m 11s
327:	learn: 0.0043645	total: 34.7s	remaining: 1m 11s
328:	learn: 0.0043645	total: 34.8s	remaining: 1m 10s
329:	learn: 0.0043645	total: 34.9s	remaining: 1m 10s
330:	learn: 0.0043645	total: 35s	remaining: 1m 10s
331:	learn: 0.0043645	total: 35s	remaining: 1m 10s
332:	learn: 0.0043645	total: 35.1s	remaining: 1m 10s

473:	learn: 0.0034333	total: 50.2s	remaining: 55.8s
474:	learn: 0.0034333	total: 50.4s	remaining: 55.7s
475:	learn: 0.0034329	total: 50.5s	remaining: 55.6s
476:	learn: 0.0034329	total: 50.6s	remaining: 55.5s
477:	learn: 0.0034329	total: 50.7s	remaining: 55.4s
478:	learn: 0.0034329	total: 50.9s	remaining: 55.4s
479:	learn: 0.0034329	total: 51s	remaining: 55.3s
480:	learn: 0.0034329	total: 51.1s	remaining: 55.1s
481:	learn: 0.0034329	total: 51.2s	remaining: 55s
482:	learn: 0.0034329	total: 51.3s	remaining: 54.9s
483:	learn: 0.0034329	total: 51.4s	remaining: 54.8s
484:	learn: 0.0034329	total: 51.4s	remaining: 54.6s
485:	learn: 0.0034328	total: 51.5s	remaining: 54.5s
486:	learn: 0.0034328	total: 51.6s	remaining: 54.3s
487:	learn: 0.0034329	total: 51.7s	remaining: 54.2s
488:	learn: 0.0034328	total: 51.9s	remaining: 54.2s
489:	learn: 0.0034328	total: 52s	remaining: 54.1s
490:	learn: 0.0034328	total: 52.1s	remaining: 54s
491:	learn: 0.0034328	total: 52.2s	remaining: 53.9s
492:	learn: 0.003432

632:	learn: 0.0034312	total: 1m 11s	remaining: 41.4s
633:	learn: 0.0034312	total: 1m 11s	remaining: 41.3s
634:	learn: 0.0034312	total: 1m 11s	remaining: 41.2s
635:	learn: 0.0034312	total: 1m 11s	remaining: 41.1s
636:	learn: 0.0034312	total: 1m 11s	remaining: 41s
637:	learn: 0.0034312	total: 1m 11s	remaining: 40.8s
638:	learn: 0.0034312	total: 1m 12s	remaining: 40.7s
639:	learn: 0.0034312	total: 1m 12s	remaining: 40.6s
640:	learn: 0.0034312	total: 1m 12s	remaining: 40.5s
641:	learn: 0.0034312	total: 1m 12s	remaining: 40.4s
642:	learn: 0.0034312	total: 1m 12s	remaining: 40.3s
643:	learn: 0.0034312	total: 1m 12s	remaining: 40.2s
644:	learn: 0.0034312	total: 1m 12s	remaining: 40.1s
645:	learn: 0.0034312	total: 1m 12s	remaining: 40s
646:	learn: 0.0034312	total: 1m 13s	remaining: 39.9s
647:	learn: 0.0034312	total: 1m 13s	remaining: 39.7s
648:	learn: 0.0034312	total: 1m 13s	remaining: 39.6s
649:	learn: 0.0034312	total: 1m 13s	remaining: 39.5s
650:	learn: 0.0034312	total: 1m 13s	remaining: 39.

790:	learn: 0.0034312	total: 1m 28s	remaining: 23.5s
791:	learn: 0.0034312	total: 1m 28s	remaining: 23.4s
792:	learn: 0.0034312	total: 1m 29s	remaining: 23.2s
793:	learn: 0.0034312	total: 1m 29s	remaining: 23.1s
794:	learn: 0.0034312	total: 1m 29s	remaining: 23s
795:	learn: 0.0034312	total: 1m 29s	remaining: 22.9s
796:	learn: 0.0034312	total: 1m 29s	remaining: 22.8s
797:	learn: 0.0034312	total: 1m 29s	remaining: 22.7s
798:	learn: 0.0034312	total: 1m 29s	remaining: 22.6s
799:	learn: 0.0034312	total: 1m 29s	remaining: 22.5s
800:	learn: 0.0034312	total: 1m 30s	remaining: 22.4s
801:	learn: 0.0034312	total: 1m 30s	remaining: 22.3s
802:	learn: 0.0034312	total: 1m 30s	remaining: 22.2s
803:	learn: 0.0034312	total: 1m 30s	remaining: 22s
804:	learn: 0.0034312	total: 1m 30s	remaining: 21.9s
805:	learn: 0.0034312	total: 1m 30s	remaining: 21.8s
806:	learn: 0.0034312	total: 1m 30s	remaining: 21.7s
807:	learn: 0.0034312	total: 1m 30s	remaining: 21.6s
808:	learn: 0.0034312	total: 1m 30s	remaining: 21.

947:	learn: 0.0034307	total: 1m 47s	remaining: 5.87s
948:	learn: 0.0034307	total: 1m 47s	remaining: 5.75s
949:	learn: 0.0034307	total: 1m 47s	remaining: 5.64s
950:	learn: 0.0034307	total: 1m 47s	remaining: 5.53s
951:	learn: 0.0034307	total: 1m 47s	remaining: 5.41s
952:	learn: 0.0034307	total: 1m 47s	remaining: 5.3s
953:	learn: 0.0034307	total: 1m 47s	remaining: 5.19s
954:	learn: 0.0034307	total: 1m 47s	remaining: 5.08s
955:	learn: 0.0034307	total: 1m 47s	remaining: 4.96s
956:	learn: 0.0034307	total: 1m 47s	remaining: 4.85s
957:	learn: 0.0034307	total: 1m 48s	remaining: 4.74s
958:	learn: 0.0034307	total: 1m 48s	remaining: 4.62s
959:	learn: 0.0034307	total: 1m 48s	remaining: 4.51s
960:	learn: 0.0034307	total: 1m 48s	remaining: 4.4s
961:	learn: 0.0034307	total: 1m 48s	remaining: 4.28s
962:	learn: 0.0034307	total: 1m 48s	remaining: 4.18s
963:	learn: 0.0034307	total: 1m 48s	remaining: 4.07s
964:	learn: 0.0034307	total: 1m 49s	remaining: 3.96s
965:	learn: 0.0034307	total: 1m 49s	remaining: 3

In [13]:
joblib.dump(cat, 'cat.pkl')

['cat.pkl']